In [1]:
import pandas as pd 
import numpy as np
import matplotlib as plt
import sklearn
import json
import ast

In [2]:
# Find ELO and Total points
df = pd.read_csv("../data/play_by_play_fbs_regular_all.csv")

In [3]:
np.random.seed(12)

In [4]:
sample = df.sample(100_000).copy()

In [5]:
df.head()

,id,offense,offense_conference,defense,defense_conference,home,away,offense_score,defense_score,game_id,...,defense_timeouts,yard_line,yards_to_goal,down,distance,scoring,yards_gained,play_type,play_text,ppa
0,252462005002,Washington,Pac-10,Air Force,Mountain West,Air Force,Washington,0,0,252462005,...,NaN,75,75,1,10,False,0,Rush,"Louis Rankin rush for 2 yards to the Wash27, c...",-0.702970
1,252462005003,Washington,Pac-10,Air Force,Mountain West,Air Force,Washington,0,0,252462005,...,NaN,73,73,2,8,False,0,Rush,"Louis Rankin rush for 6 yards to the Wash33, c...",-0.478808
2,252462005004,Washington,Pac-10,Air Force,Mountain West,Air Force,Washington,0,0,252462005,...,NaN,67,67,3,2,False,0,Rush,Isaiah Stanback rush for loss of 1 yard to the...,-0.601314
3,252462005005,Washington,Pac-10,Air Force,Mountain West,Air Force,Washington,0,0,252462005,...,NaN,68,68,4,3,False,0,Punt,"Felix Sweetman punt 49 yards to the AFA19, Jac...",NaN
4,252462005006,Air Force,Mountain West,Washington,Pac-10,Air Force,Washington,0,0,252462005,...,NaN,31,69,1,10,False,0,Pass,Shaun Carney pass complete to Greg Kirkwood fo...,-0.894781


In [6]:
df.columns

Index(['id', 'offense', 'offense_conference', 'defense', 'defense_conference',
       'home', 'away', 'offense_score', 'defense_score', 'game_id', 'drive_id',
       'drive_number', 'play_number', 'period', 'clock', 'offense_timeouts',
       'defense_timeouts', 'yard_line', 'yards_to_goal', 'down', 'distance',
       'scoring', 'yards_gained', 'play_type', 'play_text', 'ppa'],
      dtype='object')

In [7]:
team_id = pd.read_csv('../data/games.csv')

In [8]:
team_id.columns

Index(['Unnamed: 0', 'id', 'season', 'week', 'season_type', 'start_date',
       'start_time_tbd', 'neutral_site', 'conference_game', 'attendance',
       'venue_id', 'venue', 'home_id', 'home_team', 'home_conference',
       'home_points', 'home_line_scores', 'home_post_win_prob', 'away_id',
       'away_team', 'away_conference', 'away_points', 'away_line_scores',
       'away_post_win_prob', 'excitement_index'],
      dtype='object')

In [9]:
with open('../data/ELO_scores.json') as f:
  ELO_scores = json.load(f)

In [10]:
ELO_scores

{'150': 1520.7926084506182,
 '2348': 1541.1813307001223,
 '265': 1579.9925407784199,
 '2633': 1612.8512947800857,
 '2640': 1434.1006731595562,
 '21': 1583.405138482147,
 '103': 1577.7293830054923,
 '149': 1484.2474182568135,
 '259': 1644.2626183457176,
 '57': 1715.2527959869838,
 '66': 1560.7009594770534,
 '127': 1660.9069750584454,
 '194': 1843.9765954686666,
 '8': 1538.4243850897235,
 '2729': 1429.7242606091024,
 '2305': 1394.720220087898,
 '120': 1504.0988663497408,
 '62': 1479.7189831668102,
 '25': 1569.645203842333,
 '349': 1466.3981807040152,
 '218': 1532.504930969708,
 '2572': 1470.3356525847846,
 '152': 1589.378582237223,
 '55': 1452.973602384096,
 '2466': 1409.239886052264,
 '2': 1703.128925211963,
 '2641': 1562.1183959866678,
 '251': 1665.519233973493,
 '2751': 1484.1719474619963,
 '9': 1602.9162585090662,
 '30': 1693.3512902814523,
 '213': 1698.4372408018057,
 '158': 1592.9320072751955,
 '239': 1629.1406502367447,
 '98': 1497.345606351707,
 '36': 1464.3181852681498,
 '2582':

In [11]:
team_elo = {}

In [12]:
for index, row in team_id.iterrows():
    if row['home_team'] in team_elo.keys():
        pass
    else:
        team_elo[row['home_team']] = ELO_scores[str(row['home_id'])]
    if row['away_team'] in team_elo.keys():
        pass
    else:
        team_elo[row['away_team']] = ELO_scores[str(row['away_id'])]

In [13]:
team_elo

{'Louisville': 1607.0944475843905,
 'New Mexico State': 1297.3465578351627,
 'Nebraska': 1592.9320072751955,
 'TCU': 1668.6688377495695,
 'Wisconsin': 1726.2708953251051,
 'Virginia': 1542.0880615422604,
 'BYU': 1610.5696682918162,
 'Tulane': 1422.6386240672489,
 'Oklahoma': 1806.655821022029,
 'North Carolina': 1567.0349875332008,
 'Syracuse': 1527.7236672177846,
 'Georgia Tech': 1586.6720678586798,
 'Colorado': 1499.0357175447307,
 'Fresno State': 1525.8822917984603,
 'Temple': 1532.504930969708,
 'Navy': 1565.8064932804796,
 'Toledo': 1557.7679247150868,
 'Minnesota': 1593.101571910054,
 'Akron': 1348.2754661912527,
 'Ohio': 1506.2327935868434,
 'Arkansas': 1538.4243850897235,
 'UNLV': 1381.8331980704784,
 'Western Michigan': 1507.4772513911257,
 'Illinois State': 1454.5304186913572,
 'Buffalo': 1437.7573836079687,
 'Rutgers': 1454.7090720260635,
 'Central Michigan': 1457.6266567847265,
 'Eastern Kentucky': 1390.7294498528552,
 'Northern Illinois': 1543.8842143453082,
 'South Florid

In [14]:
def get_elo(play):
    return team_elo[play['offense']], team_elo[play['defense']]

In [15]:
def total_points(play):
    return play['offense_score'] + play['defense_score']

In [16]:
with open("elo_features.json", "w") as outfile:  
    json.dump(team_elo, outfile)

In [17]:
def play_type(play):
    return play['play_type']